In [14]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import tree
import graphviz
from sklearn.preprocessing import LabelEncoder

In [2]:
# Load the dataframe
df1 = pd.read_csv('DOHMH_New_York_City_Restaurant_Inspection_Results.csv', delimiter=',')
#df.head()

In [3]:
# Remove duplicates
df = df1[df1.GRADE.isin(['A','B','C'])]
df.reset_index(inplace=True)
df.drop_duplicates(['CAMIS','INSPECTION DATE', 'GRADE'], keep='last', inplace=True)
df = df[df.ZIPCODE != 7005]
#df.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [4]:
# Filling in missing boros
zips = {}
zips = {'112': 'BROOKLYN', '104':'BRONX', '100':'MANHATTAN','103':'STATEN ISLAND'}
for key in ['110', '111', '113','114','116']:
    zips[key] = 'QUEENS'
    
missing_boro = df[df.BORO == 'Missing']
first3_zip = map(lambda x:x[0:3],map(str,missing_boro.ZIPCODE))  # first 3 numbers of the zipcode
missing_boro['first3_zip'] = pd.Series(first3_zip,index=missing_boro.index)
missing_boro['BORO'] = missing_boro['first3_zip'].map(zips)
df.loc[missing_boro.index.values] = missing_boro
del missing_boro, first3_zip, zips

In [5]:
# Sorting based on date
df['GRADE DATE'] = pd.to_datetime(df['GRADE DATE'])
df['INSPECTION DATE'] = pd.to_datetime(df['INSPECTION DATE'])
df.sort_values('INSPECTION DATE',inplace=True)

In [6]:
# Find percentage of null values
for item in df.columns:
    print(item, df[item].isna().sum())

index 0
CAMIS 0
DBA 0
BORO 0
BUILDING 28
STREET 0
ZIPCODE 0
PHONE 0
CUISINE DESCRIPTION 0
INSPECTION DATE 0
ACTION 1
VIOLATION CODE 807
VIOLATION DESCRIPTION 868
CRITICAL FLAG 0
SCORE 1
GRADE 0
GRADE DATE 1
RECORD DATE 0
INSPECTION TYPE 1


In [7]:
# List the types of restaurants
out_file = 'Cuis_Descr.txt'
cuisine = []
cuisine = df.loc[:,'CUISINE DESCRIPTION'].dropna()
with open(out_file,'w') as file:
    [file.write(item + '\n') for item in cuisine.unique()]

In [8]:
df1 = df.groupby('CUISINE DESCRIPTION').count()['CAMIS']
trial = df1.sort_values(ascending=False).head(20)
print(trial.sum()*100/df1.sum())

81.71952044861258


The top 20 cuisines account for almost 82% of total values

In [9]:
# List the different types of inspections
df.loc[:,'INSPECTION TYPE'].dropna().unique()

array(['Cycle Inspection / Initial Inspection',
       'Pre-permit (Operational) / Reopening Inspection',
       'Cycle Inspection / Re-inspection',
       'Pre-permit (Operational) / Re-inspection',
       'Pre-permit (Operational) / Initial Inspection',
       'Cycle Inspection / Reopening Inspection',
       'Cycle Inspection / Compliance Inspection',
       'Pre-permit (Non-operational) / Initial Inspection'], dtype=object)

In [17]:
critical = df.loc[:,['CRITICAL FLAG', 'SCORE', 'GRADE', 'RECORD DATE', 'INSPECTION TYPE']].dropna()
for col in critical.columns[1:]:
    le = LabelEncoder()
    df[col] = le.fit_transform(critical[col])

Y = critical.iloc[:,0]
X = critical.iloc[:,1:]

ValueError: Length of values does not match length of index

In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
estimator = tree.DecisionTreeClassifier(criterion='entropy')
estimator.fit(X_train, Y_train)
Y_train = Y_train.iloc[:,0].values
Y_test = Y_test.iloc[:,0].values

ValueError: could not convert string to float: 'Cycle Inspection / Re-inspection'

# TO DO
Divide the restaurants based on types (perhaps focus on most common ones?)

In [ ]:
# Plot the distribution of scores
scores = df.loc[:,'SCORE'].dropna()
plt.hist(scores, density=True, bins=20)
plt.show()

In [ ]:
scores.describe()

In [ ]:
# Open the file with the violations
in_file = 'Violations_explanation.csv'
viol_codes = {}
with open(in_file, 'r') as f_in:
    f_in.readline() # Skip the header
    for line in f_in.readlines():
        line = line.strip('\n')
        line = line.split(';')
        viol_codes[line[0]] = line[-1]
        
codes = df['VIOLATION CODE'].map(viol_codes)
df.insert(10, 'VIOLATION TYPE', codes)
#df.head()

In [ ]:
# Plot the frequencies of violations
fig, ax = plt.subplots(figsize=(11.7, 8.27))
figure = sns.countplot(df['VIOLATION TYPE'], ax=ax)
figure.set_xticklabels(figure.get_xticklabels(),rotation=90)
_ = plt.figure()

In [ ]:
# Group by borough
df1 = df.loc[:,['BORO','SCORE']].dropna()
boros = df1.groupby(['BORO'])
boros.describe()
fig, ax = plt.subplots(figsize=(11.7, 8.27))
_ = sns.boxplot(df1['BORO'], df1['SCORE'], ax=ax)

In [ ]:
critical = df.loc[:,['CRITICAL FLAG', 'SCORE', 'GRADE', 'RECORD DATE', 'INSPECTION TYPE']].dropna()

In [ ]:
Y = critical.iloc[:,0]
X = critical.iloc[:,1:]
from sklearn.model_selection import train_test_split
from sklearn import tree

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
estimator = tree.DecisionTreeClassifier(criterion='entropy')
estimator.fit(X_train, Y_train)
Y_train = Y_train.iloc[:,0].values
Y_test = Y_test.iloc[:,0].values

NameError: name 'X' is not defined